<h1>Sommaire<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#MVP" data-toc-modified-id="MVP-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>MVP</a></span></li><li><span><a href="#Architecture-MLOps" data-toc-modified-id="Architecture-MLOps-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Architecture MLOps</a></span></li><li><span><a href="#Le-jeu-de-données" data-toc-modified-id="Le-jeu-de-données-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Le jeu de données</a></span></li></ul></li><li><span><a href="#Chargement-des-ressources" data-toc-modified-id="Chargement-des-ressources-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Chargement des ressources</a></span><ul class="toc-item"><li><span><a href="#Chargement-du-workspace" data-toc-modified-id="Chargement-du-workspace-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Chargement du workspace</a></span></li><li><span><a href="#Chargement-du-magasin-de-données" data-toc-modified-id="Chargement-du-magasin-de-données-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Chargement du magasin de données</a></span></li></ul></li><li><span><a href="#Exploration-et-analyse-des-données" data-toc-modified-id="Exploration-et-analyse-des-données-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exploration et analyse des données</a></span><ul class="toc-item"><li><span><a href="#Fichiers-clicks_hour_xxx.csv" data-toc-modified-id="Fichiers-clicks_hour_xxx.csv-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Fichiers clicks_hour_xxx.csv</a></span><ul class="toc-item"><li><span><a href="#Chargement-des-fichiers" data-toc-modified-id="Chargement-des-fichiers-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Chargement des fichiers</a></span></li><li><span><a href="#Convertion-des-types" data-toc-modified-id="Convertion-des-types-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Convertion des types</a></span></li><li><span><a href="#Longueur-des-sessions" data-toc-modified-id="Longueur-des-sessions-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Longueur des sessions</a></span></li><li><span><a href="#Indicateurs-statistiques" data-toc-modified-id="Indicateurs-statistiques-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Indicateurs statistiques</a></span><ul class="toc-item"><li><span><a href="#Variables-de-type-numérique" data-toc-modified-id="Variables-de-type-numérique-3.1.4.1"><span class="toc-item-num">3.1.4.1&nbsp;&nbsp;</span>Variables de type numérique</a></span></li><li><span><a href="#Variables-de-type-catégorie" data-toc-modified-id="Variables-de-type-catégorie-3.1.4.2"><span class="toc-item-num">3.1.4.2&nbsp;&nbsp;</span>Variables de type catégorie</a></span></li></ul></li><li><span><a href="#Répartition-des-clicks-par-article" data-toc-modified-id="Répartition-des-clicks-par-article-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span>Répartition des clicks par article</a></span></li><li><span><a href="#Répartition-des-clicks-par-utilisateur" data-toc-modified-id="Répartition-des-clicks-par-utilisateur-3.1.6"><span class="toc-item-num">3.1.6&nbsp;&nbsp;</span>Répartition des clicks par utilisateur</a></span></li><li><span><a href="#Répartition-des-clicks-par-jour" data-toc-modified-id="Répartition-des-clicks-par-jour-3.1.7"><span class="toc-item-num">3.1.7&nbsp;&nbsp;</span>Répartition des clicks par jour</a></span></li><li><span><a href="#Nombre-d'utilisateurs-par-jour" data-toc-modified-id="Nombre-d'utilisateurs-par-jour-3.1.8"><span class="toc-item-num">3.1.8&nbsp;&nbsp;</span>Nombre d'utilisateurs par jour</a></span></li></ul></li><li><span><a href="#Fichier-articles_metadata.csv" data-toc-modified-id="Fichier-articles_metadata.csv-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Fichier articles_metadata.csv</a></span><ul class="toc-item"><li><span><a href="#Convertion-des-types" data-toc-modified-id="Convertion-des-types-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Convertion des types</a></span></li><li><span><a href="#Indicateurs-statistiques" data-toc-modified-id="Indicateurs-statistiques-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Indicateurs statistiques</a></span><ul class="toc-item"><li><span><a href="#Variables-de-type-numérique" data-toc-modified-id="Variables-de-type-numérique-3.2.2.1"><span class="toc-item-num">3.2.2.1&nbsp;&nbsp;</span>Variables de type numérique</a></span></li><li><span><a href="#Variables-de-type-catégorie" data-toc-modified-id="Variables-de-type-catégorie-3.2.2.2"><span class="toc-item-num">3.2.2.2&nbsp;&nbsp;</span>Variables de type catégorie</a></span></li></ul></li><li><span><a href="#Suppression-des-variables-inutiles" data-toc-modified-id="Suppression-des-variables-inutiles-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Suppression des variables inutiles</a></span></li><li><span><a href="#Distribution-du-nombre-d'articles-par-catégorie" data-toc-modified-id="Distribution-du-nombre-d'articles-par-catégorie-3.2.4"><span class="toc-item-num">3.2.4&nbsp;&nbsp;</span>Distribution du nombre d'articles par catégorie</a></span></li><li><span><a href="#Nombre-d'articles-ajoutés-par-an" data-toc-modified-id="Nombre-d'articles-ajoutés-par-an-3.2.5"><span class="toc-item-num">3.2.5&nbsp;&nbsp;</span>Nombre d'articles ajoutés par an</a></span></li></ul></li><li><span><a href="#Fichier-articles_embeddings.pickle" data-toc-modified-id="Fichier-articles_embeddings.pickle-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Fichier articles_embeddings.pickle</a></span><ul class="toc-item"><li><span><a href="#Indicateurs-statistiques" data-toc-modified-id="Indicateurs-statistiques-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Indicateurs statistiques</a></span></li></ul></li></ul></li><li><span><a href="#Enregistrement-des-données-dans-le-feature-store" data-toc-modified-id="Enregistrement-des-données-dans-le-feature-store-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Enregistrement des données dans le feature store</a></span><ul class="toc-item"><li><span><a href="#Chargement-des-données" data-toc-modified-id="Chargement-des-données-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Chargement des données</a></span></li><li><span><a href="#Enregistrement-dans-le-datastore" data-toc-modified-id="Enregistrement-dans-le-datastore-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Enregistrement dans le datastore</a></span></li><li><span><a href="#Enregistrement-des-datasets" data-toc-modified-id="Enregistrement-des-datasets-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Enregistrement des datasets</a></span></li></ul></li><li><span><a href="#Pipeline-de-transformation-des-données" data-toc-modified-id="Pipeline-de-transformation-des-données-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Pipeline de transformation des données</a></span><ul class="toc-item"><li><span><a href="#Extraction-des-données" data-toc-modified-id="Extraction-des-données-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Extraction des données</a></span></li><li><span><a href="#Transformation-des-données" data-toc-modified-id="Transformation-des-données-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Transformation des données</a></span><ul class="toc-item"><li><span><a href="#Création-des-notation-des-articles" data-toc-modified-id="Création-des-notation-des-articles-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Création des notation des articles</a></span></li><li><span><a href="#Création-des-profils-des-articles" data-toc-modified-id="Création-des-profils-des-articles-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>Création des profils des articles</a></span></li><li><span><a href="#Création-des-profils-des-utilisateurs" data-toc-modified-id="Création-des-profils-des-utilisateurs-5.2.3"><span class="toc-item-num">5.2.3&nbsp;&nbsp;</span>Création des profils des utilisateurs</a></span></li><li><span><a href="#Exécution-du-pipeline" data-toc-modified-id="Exécution-du-pipeline-5.2.4"><span class="toc-item-num">5.2.4&nbsp;&nbsp;</span>Exécution du pipeline</a></span></li></ul></li></ul></li><li><span><a href="#Vérification-des-données" data-toc-modified-id="Vérification-des-données-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Vérification des données</a></span><ul class="toc-item"><li><span><a href="#Chargement-des-datasets" data-toc-modified-id="Chargement-des-datasets-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Chargement des datasets</a></span></li><li><span><a href="#Notation-des-articles" data-toc-modified-id="Notation-des-articles-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Notation des articles</a></span></li><li><span><a href="#Profils-des-articles" data-toc-modified-id="Profils-des-articles-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Profils des articles</a></span></li><li><span><a href="#Profils-des-utilisateurs" data-toc-modified-id="Profils-des-utilisateurs-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Profils des utilisateurs</a></span></li></ul></li></ul></div>

In [ ]:
import sys

import urllib
import zipfile

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from P9_02_scripts.datasets import *
from P9_02_scripts.data_transform.run import *
from notebook import *

# Introduction

La mission de notre entreprise est d'encourager la lecture en recommandant des contenus pertinents à nos utilisateurs.

Ce projet a pour but la réalisation d'un MVP d'un système de recommandation de contenu sous la forme d'une application mobile. Le repository Github de ce projet se trouve à l'adresse suivante : [p9-demo](https://github.com/Sako74/p9-demo).

<img src="./data/gif/mvp_demo_tiny.gif" alt="Démonstration MVP" style="width:200px;"/>
<p style="text-align: center; text-decoration: underline;">Démonstration du MVP</p>

## MVP

Fonctions de l'application mobile :
- Simulation de l’authentification d’un utilisateur.
- Recommandation de 5 articles.

Fonctions et contraintes du système de recommandation :
- Utilisation des services de Azure.
- Stockage des scripts sur Github.
- Anticiper l’ajout de nouveaux utilisateurs.
- Anticiper l’ajout de nouveaux articles.

## Architecture MLOps

Le MLOps est une culture et une pratique d'ingénierie ML qui vise à unifier le développement de systèmes ML et leur mise en opération (Ops). Appliquer le MLOps signifie que l'on vise l'automatisation et la surveillance de toutes les étapes de la construction d'un système de ML, y compris l'intégration, les tests, la publication, le déploiement et la gestion de l'infrastructure ([en savoir plus](https://cloud.google.com/architecture/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning)).

Nous avons donc mis en place une architecture MLOps afin de pouvoir itérer rapidement sur les versions de notre MVP.

<img src="./data/img/archi_mlops.png" alt="Architecture MLOps" width="900"/>
<p style="text-align: center; text-decoration: underline;">Architecture MLOps</p>

Voici la description du code couleur :
- Les blocks en gris sont ceux qui n'ont pas encore été implémentés.
- Les blocks en bleu sont ceux qui utilisent des services Azure.
- Les blocks en vert sont ceux qui utilisent Github.
- Le block en jaune est l'application mobile développée en React par un prestataire.

## Le jeu de données

Nous allons utiliser un jeu de données fournit gratuitement par [Globo.com](https://www.globo.com/), un site de news brésilien. Ce jeu de données est disponible à l'adresse suivante : [jeu de données](https://www.kaggle.com/gspmoreira/news-portal-user-interactions-by-globocom). Commençons par télécharger nos données :

In [ ]:
# Lien vers le dataset
DATASET_URL = "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+9+-+R%C3%A9alisez+une+application+mobile+de+recommandation+de+contenu/news-portal-user-interactions-by-globocom.zip"

# Fichiers du dataset
csv_files = ["articles_metadata.csv", "clicks_sample.csv"]
zip_file = "clicks"
pickle_file = "articles_embeddings.pickle"

needed_files = csv_files + [zip_file] + [pickle_file]

# Fichiers actuellement présents
current_files = os.listdir(CSV_PATH)
current_files += os.listdir(PICKLE_PATH)

# On vérifie si tous les fichiers/dossiers sont bien présents
if all([i in current_files for i in needed_files]):
    print("Tous les fichiers sont bien présents.")
# Sinon on télécharge et on extrait les données
else:
    print("Téléchargement des données en cours...")

    # On télécharge le .zip dans un fichier temporaire et on extrait les données
    tmp, _ = urllib.request.urlretrieve(DATASET_URL)
    with zipfile.ZipFile(tmp, "r") as f:
        # On extrait les fichiers csv
        for i in csv_files:
            f.extract(i, CSV_PATH)
        
        # On extrait le fichier zip contenant des fichiers csv
        with f.open(zip_file + ".zip") as f2:
            with zipfile.ZipFile(f2, "r") as f3:
                f3.extractall(CSV_PATH)
        
        # On extrait le fichier pickle
        f.extract(pickle_file, PICKLE_PATH)

    # On supprime le fichier temporaire
    urllib.request.urlcleanup()
    
    print("Téléchargement des données terminé.")

# Chargement des ressources

Nous allons charger toutes les ressources Azure qui vont nous permettre de créer et d'enregistrer des jeux de données.

## Chargement du workspace

In [ ]:
# On charge l’espace de travail Azure Machine Learning existant
ws = Workspace.from_config()

## Chargement du magasin de données

In [ ]:
# On charge le magasin de données par défaut
datastore = ws.get_default_datastore()

# Exploration et analyse des données

Cette partie représente le block `Data analysis` de notre architecture.

## Fichiers clicks_hour_xxx.csv

Le jeu de données est composé de fichiers de logs enregistrés heure par heure. Ils contiennent des informations sur les articles cliqués par les utilisateurs ainsi que des informations sur les sessions des utilisateurs.

### Chargement des fichiers

In [ ]:
clicks_dir = CSV_PATH + "clicks"

# On ouvre les fichiers et on ajoute les données dans une liste
clicks = []
for i in tqdm(os.listdir(clicks_dir), leave=False):
    tmp = pd.read_csv(os.path.join(clicks_dir, i))
    clicks.append(tmp)
    
# On concatène toutes les données
clicks = pd.concat(clicks)
clicks.shape

Nous avons en tout 2.988.181 individus et 12 variables. Voyons un échantillon de ces données :

In [ ]:
clicks.head()

### Convertion des types

Nous allons commencer par convertir les types des variables. Certains types, comme `uint8`, ont été choisis afin d'optimiser la mémoire. Nous les avons sélectionné après avoir observé les indicateurs statistiques de ces variables.

In [ ]:
# On met à jour le type des variables
clicks = clicks.astype({
    "user_id": np.uint64,
    "session_id": np.uint64,
    "session_start": np.uint64,
    "session_size": np.uint16,
    "click_article_id": np.uint64,
    "click_timestamp": np.uint64,
    "click_environment": np.uint8,
    "click_deviceGroup": np.uint8,
    "click_os": np.uint8,
    "click_country": np.uint8,
    "click_region": np.uint8,
    "click_referrer_type": np.uint8
})

In [ ]:
# On convertit les timestamps en datetime
clicks["session_start"] = pd.to_datetime(clicks['session_start'], unit='ms')
clicks["click_timestamp"] = pd.to_datetime(clicks['click_timestamp'], unit='ms')

In [ ]:
# On renomme les colonnes
clicks = clicks.rename(columns={
    "session_start": "session_start_dt",
    "click_timestamp": "click_dt",
})

In [ ]:
# On trie par ordre chronologique
clicks = clicks.sort_values("click_dt")

### Longueur des sessions

Voyons quelques indicateurs statistiques sur la longueur des sessions :

In [ ]:
# On regroupe les données par session
grp = clicks.groupby("session_id")

# On agrège les données
grp_agg = grp.agg({"session_start_dt": "min", "click_dt": "max"})

# On calcule le timedelta entre le dernier click et le début de la session
grp_agg["session_length_td"] = grp_agg["click_dt"] - grp_agg["session_start_dt"]
grp_agg["session_length_td"].describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99])

99% des sessions durent moins de 5h40 et 75% durent moins de 16min.

La session la plus longue a duré 28 jours, il s'agit donc d'une valeur atypique.

### Indicateurs statistiques

In [ ]:
num_vars = [
    "session_size",
    "session_start_dt",
    "click_dt"
]

cat_vars = [
    "user_id",
    "session_id",
    "click_article_id",
    "click_environment",
    "click_deviceGroup",
    "click_os",
    "click_country",
    "click_region",
    "click_referrer_type"
]

#### Variables de type numérique

Voici le tableau des indicateurs statistiques des variables de type numérique :

In [ ]:
clicks[num_vars].describe(
    percentiles=[0.25, 0.5, 0.75, 0.95, 0.99],
    datetime_is_numeric=True
).T

En moyenne, les utilisateurs cliquent sur 4 articles par session.

99% des clicks ont été effectués avant le 16/10/2017 à 23h.

#### Variables de type catégorie

Voici le tableau des indicateurs statistiques des variables de type catégorie :

In [ ]:
clicks[cat_vars].astype("category").describe().T

Il y a 322.897 utilisateurs différents qui ont cliqués sur 46.033 articles différents.

Un des utilisateurs a cliqué sur 1232 articles.

Un des articles a été cliqué 37213 fois.

### Répartition des clicks par article

In [ ]:
# On regroupe les données par article
grp = clicks.groupby("click_article_id")

# On agrège les données
grp_agg = grp.agg({"click_dt": "count"})
grp_agg = grp_agg.rename(columns={"click_dt": "click_nb"})

# On affiche les indicateurs statistiques
grp_agg.describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99]).T

99% des articles ont été cliqués moins de 1204 fois et 75% moins de 6 fois.

L'article qui a été cliqué 37213 fois est une valeur atypique. Il s'agit peut-être d'un article qui a fait le buzz...

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

grp_agg.plot.hist(bins=50, ax=ax)

ax.set_title("Distribution des clicks par article")
ax.set_xlabel("Nombre de clicks par article")
ax.set_ylabel("Effectif (log)")
ax.set_yscale("log")

plt.show()

### Répartition des clicks par utilisateur

In [ ]:
# On regroupe les données par article
grp = clicks.groupby("user_id")

# On agrège les données
grp_agg = grp.agg({"click_dt": "count"})
grp_agg = grp_agg.rename(columns={"click_dt": "click_nb"})

# On affiche les indicateurs statistiques
grp_agg.describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99]).T

99% des articles ont été cliqués moins de 67 fois et 75% moins de 10 fois.

L'utilisateur qui a été cliqué 1232 fois est une valeur atypique.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

grp_agg.plot.hist(bins=50, ax=ax)

ax.set_title("Distribution des clicks par utilisateur")
ax.set_xlabel("Nombre de clicks par utilisateur")
ax.set_ylabel("Effectif (log)")
ax.set_yscale("log")

plt.show()

### Répartition des clicks par jour

In [ ]:
# On regroupe les données par jour
grp = clicks.groupby(pd.Grouper(key="click_dt", freq="D"))

# On agrège les données
grp_agg = grp.agg({
    "click_dt": lambda x: x.max() - x.min(),
    "click_article_id": "count"
})
grp_agg = grp_agg.rename(columns={
    "click_dt": "click_delta",
    "click_article_id": "click_nb",
})
grp_agg

On constate qu'à partir du 17/10 il y a beaucoup moins de cliques par jours.

Afin de conserver un nombre de cliques par jour conséquent, nous n'allons travailler que sur la période du 01/10 au 16/10 :

In [ ]:
# On supprime les jours qui ont trop peu de cliques
clicks = clicks[clicks["click_dt"] < datetime(2017, 10, 17)]

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

clicks["click_dt"].dt.date.value_counts().sort_index().plot.bar(ax=ax)

ax.set_title("Nombre de clicks par jour")
ax.set_xlabel("Jour")
ax.set_ylabel("Nombre de clicks")

plt.show()

Nous avons au minimum 92.163 cliques dans une journée. Il s'agit d'un nombre conséquent et nous pourrons donc splitter nos données en fonction des jours.

### Nombre d'utilisateurs par jour

Les systèmes de recommandation sont confrontés au cold start problem. Par exemple, quoi recommander à un nouvel utilisateur dont on a que très peu d'information ?

Nous allons donc observer l'évolution du nombre de nouveaux utilisateurs jour par jour :

In [ ]:
# On regroupe les données par jour
grp = clicks.groupby(pd.Grouper(key="click_dt", freq="D"))

# On calcule l'ensemble cumulé des ids des utilisateurs par jour
grp_agg = grp.agg({"user_id": lambda x: x.unique().tolist()}) 
grp_agg["user_id"] = grp_agg["user_id"].cumsum().apply(set)

# On ajoute une colonne représentant le jour précédent
grp_agg["prev_user_id"] = grp_agg["user_id"].shift(1)
grp_agg["prev_user_id"] = grp_agg["prev_user_id"].fillna("").apply(set)

# On calcule le nombre d'utilisateur connus et inconnus par jour
grp_agg["user_nb"] = grp_agg[["user_id", "prev_user_id"]].aggregate(lambda x: len(x[0] & x[1]), axis=1)
grp_agg["new_user_nb"] = (grp_agg["user_id"] - grp_agg["prev_user_id"]).apply(len)

# On nettoie les colonnes et l'index pour l'affichage
grp_agg = grp_agg[["user_nb", "new_user_nb"]]
grp_agg["%"] = grp_agg["new_user_nb"] / (grp_agg["new_user_nb"] + grp_agg["user_nb"])
grp_agg = grp_agg.rename(columns={
    "user_nb": "nombre d'utilisateurs connus dans la journée",
    "new_user_nb": "nombre d'utilisateurs inconnus dans la journée",
    "%": "ratio d'inconnus",
})
grp_agg.index = grp_agg.index.date
grp_agg

Le premier jour, tous les utilisateurs sont considérés comme inconnus.

Au bout de 5 jours, nous avons déjà moins de 10% d'utilisateurs inconnus.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

grp_agg[[
    "nombre d'utilisateurs connus dans la journée",
    "nombre d'utilisateurs inconnus dans la journée"
]].plot.bar(ax=ax)

ax.set_title("Nombre d'utilisateurs par jour avec mise à jour quotidienne des nouveaux utilisateurs")
ax.set_xlabel("Jour")
ax.set_ylabel("Nombre d'utilisateurs")

plt.show()

In [ ]:
# On nettoie les ressources
del(clicks)
gc.collect()

## Fichier articles_metadata.csv

Ce fichier contient des informations sur les articles.

In [ ]:
# On récupère les métadonnées des articles
articles_metadata = pd.read_csv(CSV_PATH + "articles_metadata.csv")
articles_metadata.shape

Nous avons 364.047 individus et 5 variables. Voyons un échantillon de ces données :

In [ ]:
articles_metadata.head()

### Convertion des types

Nous allons commencer par convertir les types des variables. Certains types, comme `uint8`, ont été choisis afin d'optimiser la mémoire. Nous les avons sélectionné après avoir observé les indicateurs statistiques de ces variables.

In [ ]:
# On met à jour le type des variables
articles_metadata = articles_metadata.astype({
    "article_id": np.uint64,
    "category_id": np.uint16,
    "created_at_ts": np.uint64,
    "publisher_id": np.uint8,
    "words_count": np.uint16
})

In [ ]:
# On convertit les timestamps en datetime
articles_metadata["created_at_ts"] = pd.to_datetime(articles_metadata['created_at_ts'], unit='ms')

In [ ]:
# On renomme les colonnes
articles_metadata = articles_metadata.rename(columns={
    "created_at_ts": "created_dt",
    "words_count": "word_nb",
})

### Indicateurs statistiques

In [ ]:
num_vars = ["word_nb", "created_dt"]
cat_vars = ["article_id", "category_id", "publisher_id"]

#### Variables de type numérique

Voici le tableau des indicateurs statistiques des variables de type numérique :

In [ ]:
articles_metadata[num_vars].describe(
    percentiles=[0.25, 0.5, 0.70, 0.95, 0.99],
    datetime_is_numeric=True
).T

En moyenne, les articles sont composés de 191 mots. Il s'agit donc d'articles très courts.

70% ont été créés avant le 13/10/2017 et ont donc potentiellement été visibles par les utilisateurs de notre jeu de données.

#### Variables de type catégorie

Voici le tableau des indicateurs statistiques des variables de type catégorie :

In [ ]:
articles_metadata[cat_vars].astype("category").describe().T

Il y a 461 catégories d'articles différentes.

`publisher_id` ne contient qu'une seule valeur, on pourra donc supprimer cette variable.

### Suppression des variables inutiles

In [ ]:
# On supprime les colonnes inutiles
articles_metadata = articles_metadata.drop(columns=["publisher_id"])

### Distribution du nombre d'articles par catégorie

In [ ]:
# On regroupe les données par catégorie
grp = articles_metadata.groupby("category_id")

# On agrège les données
grp_agg = grp.agg({"article_id": "count"})
grp_agg = grp_agg.rename(columns={"article_id": "article_nb"})

# On calcule le ratio du nombre d'articles
grp_agg["ratio"] = grp_agg["article_nb"] / grp_agg["article_nb"].sum()

# On affiche les indicateurs statistiques
grp_agg.describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99]).T

Plus de 50% des catégories ont au moins 36 articles et plus de 25% ont au moins 520 articles.

Une des catégorie contient 12.817 articles, soit 3.5% du total des articles.

In [ ]:
# On trie par nombre d'articles
grp_agg = grp_agg.sort_values("article_nb", ascending=False)

# On prend les catégories les plus représentées
most_frequents = grp_agg.iloc[:50]

# On regroupe les autres
others = grp_agg.iloc[50:].sum().to_frame().T
others.index = ["others"]

# On concatène les données
grp_agg = pd.concat([most_frequents, others])

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

most_frequents[["article_nb"]].plot.bar(ax=ax)

ax.set_title("Répartition du nombre d'articles des 50 catégories les plus représentées")
ax.set_xlabel("Catégorie")
ax.set_ylabel("Nombre d'articles")

plt.show()

On constate que les articles sont bien répartis dans de nombreuses catégories. Il pourrait s'agir d'une feature intéressante pour les modèles de type content based.

### Nombre d'articles ajoutés par an

In [ ]:
# On regroupe les données par année
grp = articles_metadata.groupby(pd.Grouper(key="created_dt", freq="Y"))

# On calcule le nombre d'articles ajoutés
grp_agg = grp.agg({"article_id": "count"})

# On calcule le ratio du nombre d'articles
grp_agg["ratio"] = grp_agg["article_id"] / grp_agg["article_id"].sum()

# On nettoie les colonnes et l'index pour l'affichage
grp_agg = grp_agg.rename(columns={
    "article_id": "nombre d'articles ajoutés dans l'année"
})
grp_agg.index = grp_agg.index.year
grp_agg

43% des articles ont été ajouté en 2017.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

grp_agg[["nombre d'articles ajoutés dans l'année"]].plot.bar(ax=ax)

ax.set_title("Nombre d'articles ajoutés par an")
ax.set_xlabel("Année")
ax.set_ylabel("Nombre d'articles")

plt.show()

In [ ]:
# On nettoie les ressources
del(articles_metadata)
gc.collect()

## Fichier articles_embeddings.pickle

Les contenus des articles ont été transformés en embeddings par le block ACR (Article Content Representation) d'un model de recommandation nommé CHAMELEON ([en savoir plus](https://arxiv.org/abs/1904.10367)). Cette représentation numérique dans un espace à dimensions réduites va nous premettre de calculer des score de similarité entre les articles. Ces scores sont notamment utilisés par les modèles de type content based.

In [ ]:
# On récupère les embeddings des articles
with open(PICKLE_PATH + "articles_embeddings.pickle", "rb") as f:
    articles_embeddings = pickle.load(f)
    
# On les place dans un dataframe
articles_embeddings = pd.DataFrame(
    articles_embeddings.tolist(),
    columns=[f"emb_{i}" for i in range(articles_embeddings.shape[1])]
)
articles_embeddings.shape

Les articles sont représentés par des embeddings de dimension 250.

In [ ]:
articles_embeddings.head()

### Indicateurs statistiques

In [ ]:
articles_embeddings.iloc[:10].T.describe().T

En observant les indicateurs statistiques des embeddings des 10 premiers articles, on s'apperçoit que les embeddings ont des valeurs comprises entre -1 et 1, qu'ils sont centrés en 0 et ont un écart type de 0.5.

In [ ]:
# On nettoie les ressources
del(articles_embeddings)
gc.collect()

# Enregistrement des données dans le feature store

A terme, nous souhaiterions obtenir l'architecture suivante pour la collecte des données :

<img src="./data/img/archi_data_collect.png" alt="Acquisition des données" style="width:600px;"/>
<p style="text-align: center; text-decoration: underline;">Acquisition des données</p>

Nous y retrouvons les blocs suivantes :
- `Mobile app` :
	- L’utilisateur interagit avec l’application (par exemple : clique sur un article).
	- L’application envoie ces informations au backend.
	- Permet l’ajout de nouveaux utilisateurs via un formulaire d'inscription (non implémenté).

- `Dashboard` (non implémenté) :
	- Permet l’ajout de nouveaux articles via un formulaire d'inscription.

- `App backend` (non implémenté) :
	- Enregistre les évènements de l’application mobile dans des logs (comme les fichiers clicks_hour_xxx.csv).
	- Permet l’ajout de nouveaux utilisateurs.
	- Permet l’ajout de nouveaux articles.

- `ETL` (non implémenté) :
	- Azure function qui s’exécute par exemple 1 fois par jour.
	- Extract, Transform and Load des données du backend vers le feature store (création des embeddings etc...).

- `Feature store`:
    - Azure Blob Datastore pour l'enregistrement des fichiers.
    - Azure Dataset pour regrouper les fichiers enregistrés et les rendre disponibles pour la transformation des données.
	- Enregistrement des features qui seront utilisés par les modèles de recommandation en développement et en production.
    
Les blocs en gris ne sont pas encore implémentés. Nous allons donc manuellement enregistrer notre jeu de données dans le feature store.

## Chargement des données

On commence par charger le contenu des fichiers `clicks_hour_xxx.csv` dans un DataFrame via la fonction `get_clicks`. Cette fonction va automatiquement réaliser toutes les étapes de nettoyage que l'on a effectué durant l'exploration et l'analyse des données.

In [ ]:
# On charge les données
clicks = get_clicks(CSV_PATH + "clicks")

# On supprime les jours qui ont trop peu de cliques (vu dans l'EDA)
clicks = clicks[clicks["click_dt"] < datetime(2017, 10, 17)]

In [ ]:
clicks.iloc[[0, 1, -2, -1]]

De même, on fait appel à la fonction `get_articles` qui va charger les metadatas des articles, les nettoyer et les concaténer avec les embeddings.

In [ ]:
articles = get_articles(
    CSV_PATH + "articles_metadata.csv",
    PICKLE_PATH + "articles_embeddings.pickle",
)

In [ ]:
articles.iloc[[0, 1, -2, -1]]

## Enregistrement dans le datastore

On enregistre les données des cliques dans des fichiers séparés en les regroupant jour par jour.

On remarquera, en observant les logs ci-dessous, que l'on a créé un arborescence de type `<année>/<mois>/<jour>`. En effet, Azure propose une fonctionnalité qui permettra par la suite de rechercher et de filtrer les données des Datasets en prenant en compte cette arborescence ([en savoir plus](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.dataset_factory.tabulardatasetfactory?view=azure-ml-py#from-parquet-files-path--validate-true--include-path-false--set-column-types-none--partition-format-none-)). Dans la pratique, cette fonctionnalité semble ralentir le chargement des Datasets.

Nous avons finalement choisi une autre fonctionnalité proposé par Azure et qui permet de filter des données des Datasets en se basant sur une variable de type horodatage présente dans les fichiers ([en savoir plus](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py#with-timestamp-columns-timestamp-none--partition-timestamp-none--validate-false----kwargs-)).

In [ ]:
# On enregistre les données jour par jour dans le datastore
upload_clicks_in_datastore(clicks, datastore)

On enregistre ensuite les données des articles.

In [ ]:
# On enregistre les données année par année dans le datastore
upload_articles_in_datastore(articles, datastore)

## Enregistrement des datasets

Un Dataset Azure est une abstraction qui nous permet de créer un jeu de données en regroupant de manière symbolique des fichiers du Datastore. Ainsi, on pourra continuer d'ajouter des données de cliques sous forme de fichiers dans le Datastore et il nous suffira juste de mettre à jour le Dataset correspondant. De plus, chaque mise à jour d'un Dataset est versionnée, on conservera ainsi l'historique de l'évolution des Datasets.

La fonction suivante va regouper tous les fichiers des données de cliques disponibles sur le Datastore et va créer/updater le Dataset `clicks`.

In [ ]:
# On crée/update le dataset avec tous les fichiers clicks présents dans le datastore
clicks_ds = create_update_clicks_dataset(ws, datastore)

La fonction suivante va regouper tous les fichiers des données des articles disponibles sur le Datastore et va créer/updater le Dataset `articles`.

In [ ]:
# On crée/update le dataset avec tous les fichiers articles présents dans le datastore
articles_ds = create_update_articles_dataset(ws, datastore)

# Pipeline de transformation des données

Nous allons devoir transformer nos données afin qu'elles soient utilisables par nos modèles. Cette section représente les blocs `Data extraction` et `Data transform` de notre architecture. On utilisera les mêmes scripts durant l'expérimentation des modèles que durant le déploiement du modèle final en production.

## Extraction des données

La première étape du pipeline consiste à extraire les données du feature store et de les spliter en 1 jeu d'entrainement, 1 jeu de validation et 1 jeu de test.

Le paramètre `test_end_dt` correspond à l'horodatage maximal qui sera présent dans le jeu de test.
`test_day_nb` représente le nombre de jour que l'on prendra dans le jeu de test.
On mettra ensuite les `valid_day_nb` jours précédents dans le jeu de validation.
Enfin, on prendra les `train_day_nb` jours précédents dans le jeu d'entrainement.

Nous allons donc spliter nos données de manière chronologique. Cela évitera d'entrainer notre modèle avec des données qu'il n'est pas censé voir, ce qui pourrait fausser son évaluation.

In [ ]:
# On crée l'étape d'extraction des données du feature store
(
    data_extraction_step,
    test_clicks_path,
    valid_clicks_path,
    train_clicks_path,
    articles_path
) = get_data_extraction_step(
    ws,
    SCRIPTS_PATH + "data_transform",
    test_end_dt=datetime(2017, 10, 17),
    test_day_nb=1,
    valid_day_nb=1,
    train_day_nb=5
)

## Transformation des données

Nous allons effectuer du feature engineering afin d'adapter nos données aux besoin de nos modèles.

### Création des notation des articles

Les modèles de collaborative filtering ont besoin de notes représentant les affinités que les utilisateurs ont pour les articles.

Nous n'avons pas de note explicite dans notre jeu de données. Nous allons donc devoir créer des notes inplicites, déduite à partir du nombre de clicks sur chaque article. Nous allons créer 2 notes différentes qui seront testées durant la recherche des hyperparamètres :
- `rating_click_nb` : Nombre total de clicks par un utilisateur sur un article.
- `rating_click_per_session_ratio` : `rating_click_nb` normalisé par le nombre total de clicks dans la session de l'utilisateur.

Le jeu de données original ne contient pas non plus de vrais négatif (par exemple : des articles qui ont été vus par l'utilisateur mais sur lesquels il n'a pas cliqué). Nous allons donc ajouter des vrais négatifs en prenant au hasard des articles du jeu de données `clicks_path` et qui n'ont jamais été cliqués par l’utilisateur. Le paramètre `tn_nb` permet de spécifier le nombre de vrais négatif à sélectionner pour chaque utilisateur. Si ce paramètre vaut `None`, la fonction calcule `tn_nb` afin de créer un jeu de données équilibré.

In [ ]:
# On crée les étapes de notation des articles pour chacun des jeux de données
test_user_article_ratings_step = get_user_article_ratings_step(
    ws,
    SCRIPTS_PATH + "data_transform",
    clicks_path=test_clicks_path,
    tn_nb=100,
    dataset_name="test_user_article_ratings"
)

valid_user_article_ratings_step = get_user_article_ratings_step(
    ws,
    SCRIPTS_PATH + "data_transform",
    clicks_path=valid_clicks_path,
    tn_nb=100,
    dataset_name="valid_user_article_ratings"
)

train_user_article_ratings_step = get_user_article_ratings_step(
    ws,
    SCRIPTS_PATH + "data_transform",
    clicks_path=train_clicks_path,
    tn_nb=None,
    dataset_name="train_user_article_ratings"
)

### Création des profils des articles

Pour chaque article, nous allons créer un profil qui va permettre de la caractériser. Nous allons principalement reprendre les données du Dataset `articles`. Nous allons aussi créer des features par agrégation de données (nombre total de clicks etc…).

Ces données seront principalement utilisées par les modèles de type content based.

In [ ]:
# On crée l'étape de création des profils des articles
article_profiles_step, article_profiles_path = get_article_profiles_step(
    ws,
    SCRIPTS_PATH + "data_transform",
    clicks_path=train_clicks_path,
    articles_path=articles_path,
    dataset_name="article_profiles"
)

### Création des profils des utilisateurs

Comme précédemment, nous allons créer un profil par utilisateur. Nous pourrons y ajouter des données spécifiques aux utilisateurs qui vont permettre de rechercher les utilisateurs similaires. Nous allons aussi agréger les données des articles cliqués par l'utilisateur afin de créer le profil d'article moyen de l'utilisateur. On pourra alors utiliser ces données dans un modèle de type content based pour trouver des articles similaires à proposer à l'utilisateur.

In [ ]:
# On crée l'étape de création des profils des utilisateurs
train_user_profiles_step = get_user_profiles_step(
    ws,
    SCRIPTS_PATH + "data_transform",
    clicks_path=train_clicks_path,
    article_profiles_path=article_profiles_path,
    dataset_name="train_user_profiles"
)

### Exécution du pipeline

On peut maintenant lancer l'exécution du pipeline complet sur des clusters de calcul. On pourra aussi lancer ce même pipeline via le workflow [Transform data](https://github.com/Sako74/p9-demo/actions/workflows/data_transform.yml) des Github actions.

<img src="./data/img/data_transform.png" alt="Pipeline de transformation des données" style="width:900px;"/>
<p style="text-align: center; text-decoration: underline;">Pipeline de transformation des données</p>

In [ ]:
# On soummet l'exécution du pipeline
run = exp_submit(
    ws,
    steps=[
        data_extraction_step,
        test_user_article_ratings_step,
        valid_user_article_ratings_step,
        train_user_article_ratings_step,
        article_profiles_step,
        train_user_profiles_step
    ],
    regenerate_outputs=False,
    wait_for_completion=False,
    show_output=False
)

# Vérification des données

Nous allons charger les Datasets précédemment générés comme le ferait un script d'entrainement de modèle.

Nous n'allons ici que visualiser des échantillons de données afin de vérifier s'il n'y a pas d'erreur flagrante. A terme, il serait intéressant de vérifier l'intégrité des données automatiquement. Cette étape correspondrait aux blocs `Data verification` de notre architecture.

## Chargement des datasets

In [ ]:
# On charge les datasets
test_user_article_ratings_ds = Dataset.get_by_name(ws, "test_user_article_ratings")
valid_user_article_ratings_ds = Dataset.get_by_name(ws, "valid_user_article_ratings")
train_user_article_ratings_ds = Dataset.get_by_name(ws, "train_user_article_ratings")

article_profiles_ds = Dataset.get_by_name(ws, "article_profiles")
train_user_profiles_ds = Dataset.get_by_name(ws, "train_user_profiles")

In [ ]:
# On charge les datasets dans des DataFrames
test_user_article_ratings = test_user_article_ratings_ds.to_pandas_dataframe()
valid_user_article_ratings = valid_user_article_ratings_ds.to_pandas_dataframe()
train_user_article_ratings = train_user_article_ratings_ds.to_pandas_dataframe()

article_profiles = article_profiles_ds.to_pandas_dataframe()
train_user_profiles = train_user_profiles_ds.to_pandas_dataframe()

## Notation des articles

In [ ]:
test_user_article_ratings.shape

In [ ]:
test_user_article_ratings.iloc[[0, 1, -2, -1]]

Les données des notations des articles du jeu de test semblent cohérentes. Les valeurs `ERROR` correspondent aux horodatages inconnus de vrais négatifs que l'on a ajouté. Cela n'affectera pas l'entrainement des modèles. 

In [ ]:
valid_user_article_ratings.shape

In [ ]:
valid_user_article_ratings.iloc[[0, 1, -2, -1]]

Les données des notations des articles du jeu de validation semblent cohérentes. Les valeurs `ERROR` correspondent aux horodatages inconnus de vrais négatifs que l'on a ajouté. Cela n'affectera pas l'entrainement des modèles. 

Les données semblent cohérentes.

In [ ]:
train_user_article_ratings.shape

In [ ]:
train_user_article_ratings.iloc[[0, 1, -2, -1]]

Les données des notations des articles du jeu d'entrainement semblent cohérentes. Les valeurs `ERROR` correspondent aux horodatages inconnus de vrais négatifs que l'on a ajouté. Cela n'affectera pas l'entrainement des modèles. 

## Profils des articles

In [ ]:
article_profiles.shape

In [ ]:
article_profiles.iloc[[0, 1, -2, -1]]

Les données des profils des articles semblent cohérentes.

## Profils des utilisateurs

In [ ]:
train_user_profiles.shape

In [ ]:
train_user_profiles.iloc[[0, 1, -2, -1]]

Les données des profils des utilisateurs semblent cohérentes.